# Read BVH file ✓

In [ ]:
from Motion.BVH import load

animation, joints_names, frametime = load("./test.bvh")

# Write BVH file ✓

In [ ]:
from Motion.BVH import save

save(filename="test.bvh", anim=animation, names=joints_names, frametime=frametime)

# Edit BVH file (center animation) ✓

In [ ]:
from Motion.BVH import load, save

animation, joints_names, frametime = load("./03_LPDI_retargeted/Electron_Phrase_01_000001_001800.bvh")
# animation.positions[:, :, [0, 2]] -= animation.positions[0, 0, [0, 2]]
# animation.positions[:, 0] -= [0, 80, 0]
save(
    filename="centered.bvh",
    anim=animation,
    names=joints_names,
    frametime=frametime,
    order="xyz",
)

Rotate BVH file from Z-up to Y-up (I think this works!!)

In [ ]:
from Motion.BVH import load, save
from Motion.Quaternions import Quaternions
import numpy as np

animation, joints_names, frametime = load("./03_LPDI_retargeted/RC_SET_A_01_005401_005803.bvh", ignore_leaf_bones=True, world=False)

# Rotations: rotate the root bone by 90
rotation = Quaternions.from_euler(np.radians(np.array([-90, 0, 0])), order="xyz")
root_rot = animation.rotations[:, 0]
root_rot = root_rot * rotation
animation.rotations[:, 0] = root_rot

# Positions
positions_y_up = np.copy(animation.positions[:, 0])
positions_y_up[..., 1] = animation.positions[:, 0, 2]
positions_y_up[..., 2] = -animation.positions[:, 0, 1]
animation.positions[:, 0] = positions_y_up

save(
    filename="rotated_Yup.bvh",
    anim=animation,
    names=joints_names,
    frametime=frametime,
    order="yzx",
)

In [ ]:
animation.positions.shape

# Convert BVH data to XYZ positions ✓

In [ ]:
from Motion.Animation import positions_global
from Motion.BVH import load

animation, joints_name, frametime = load("./test.bvh")
anim_xyz = positions_global(animation)
print(anim_xyz)

# Get kinematic chain from BVH ✓

In [ ]:
from Motion.BVH import load
from Motion.AnimationStructure import get_kinematic_chain

animation, joints_name, frametime = load("./test.bvh")
kinematic_chain = get_kinematic_chain(animation.parents)
print(kinematic_chain)

# Visualize motion ✓

In [ ]:
from Motion.Animation import positions_global
from Motion.AnimationStructure import get_kinematic_chain
from Motion.BVH import load
from Motion.plot_script import plot_3d_motion


animation, joints_name, frametime = load("./test.bvh")
skeleton = get_kinematic_chain(animation.parents)
anim_xyz = positions_global(animation)

plot_3d_motion("test_vis.mp4", skeleton, anim_xyz[:600], title="Test viz", fps=100)

# Convert angles to various representations

Be careful with the order of the axes in the BVH file you're using!!

In [ ]:
import torch
import Motion.transforms as tr
from Motion.Animation import Animation
from Motion.BVH import load, save
from Motion.Quaternions import Quaternions

order = "zyx"
animation, joints_names, frametime = load("test.bvh")
n_frames, n_joints = animation.shape
print(f"Animation loaded\nn frames: {n_frames}\tn_joints: {n_joints}")

# animation.rotations is by default represented as Quaternions
rotations = torch.tensor(animation.rotations.qs)
# print("Quaternions shape:", rotations.shape)

# # # Convert quaternions to 6D representation...
# rotations = tr.quat2repr6d(rotations)
# print("Repr6d shape:", rotations.shape)

# # ... and back to quaternions
# rotations = tr.repr6d2quat(rotations)

# # and now to Euler angles
# rotations = tr.quat2euler(rotations, order="xyz", degrees=False)
# print("Euler shape:", rotations.shape)

# Now export back to BVH... and it should be the exact same :)
new_anim = Animation(
    animation.rotations,
    animation.positions,
    animation.orients,
    animation.offsets,
    animation.parents,
)
save("test_angular_conversions.bvh", animation, joints_names, frametime)

# Export to and import from SMPL format ✓

**!! This only works with Y-up convention!**

PKL -> BVH ✓

In [ ]:
from pathlib import Path
from Motion.smpl import load_smpl, smpl_to_bvh_data
from Motion.smpl_utils.utils import bvh

smpl_file = "chopin_gens/NocturneB49_v1.pkl"
smpl_data = load_smpl(smpl_file)
smpl_data["smpl_trans"] *= 100
bvh_data = smpl_to_bvh_data(smpl_data, frametime=1/30)
bvh.save(f"{Path(smpl_file).stem}_converted.bvh", bvh_data)

BVH -> PKL -> BVH ✓

In [ ]:
from Motion.smpl import smpl_to_bvh_data, bvh_data_to_smpl
from Motion.smpl_utils.utils import bvh

bvh_file = "edge_aistpp_converted.bvh"
bvh_data = bvh.load(bvh_file)

smpl_data = bvh_data_to_smpl(bvh_data)
backtobvhdata = smpl_to_bvh_data(smpl_data, frametime=1 / 60)
bvh.save("tothefutureandback.bvh", backtobvhdata)

To do:
 - [ ] Forward Kinematics :) 
 - [ ] Unify everything. For instance, transforms should accept numpy arrays and not just torch tensors and it should be possible to convert directly from SMPL to Animation object and back.